# Identifier les textes évoquant le concept de république

In [47]:
import pandas as pd
import re
import csv

df = pd.read_csv(
    "../data/interim/data_cleaning.csv", low_memory=False, dtype={"ID_orateur": str}
)
df.shape

(215127, 59)

## INTRODUIRE PRÉ-TRAITEMENT TEXTE

Pour simplifier la vie et faciliter aussi possibles perf d'un futur modèle, virer les parenthèses et balises ici pour s'économiser pas mal de choses du côté des noms de groupes

In [48]:
# nettoyage moche du texte


def nettoyer_texte(texte):
    if not isinstance(texte, str):
        return texte
    # Supprimer les balises HTML/XML
    texte = re.sub(r"<[^>]+>", "", texte)
    # Supprimer contenu entre parenthèses ou crochets
    texte = re.sub(r"\([^)]*\)", "", texte)
    # # TODO: aviser si enlève crochets, ou juste […] [...], surtout des troncatures citations
    # texte = re.sub(r"\[[^\]]*\]", "", texte)
    # Supprimer les espaces multiples
    texte = re.sub(r"\s+", " ", texte).strip()
    return texte


df["Texte_clean"] = df["Texte"].apply(nettoyer_texte)

## Regex

Logique de la tentative :
- regex
- mais exclure certains termes
- mais comme les termes exclus peuvent apparaitre aussi avec les termes voulu, éviter de chainer et finir par virer des trucs qu'on aurait voulu (les idées républicaines sont menacées par Les Républicains)

In [49]:
# Regex du champ lexical République (simplifié ici)

pattern_lexical = re.compile(
    r"républi",
    re.I,
)

# Regex des expressions à exclure
# ici garder la casse
pattern_excl = re.compile(
    r"\b[Ll]es Républicains\b"
    r"|\b[Pp]résident[s]? de la République\b"
    r"|\b[Pp]résidence[s]? de la République\b"
    r"|\b[Gg]auche démocrate et républicaine" # pas de fin de mot car aussi -NUPES
    r"|\b[Rr]épublique en marche\b"
    r"|\b[Pp]rocureur[s]? de la République\b"
    r"|\b[Cc]our[s]? de justice de la République\b"
    r"|\b[Aa]dministration générale de la République\b"
    # r"|" + pattern_pays  # ajout des exclusions de pays si existe
)


def contains_lexical_outside_excl(text):
    # Trouve toutes les positions des expressions exclues
    excl_positions = [m.span() for m in pattern_excl.finditer(text)]

    # Fonction pour vérifier si une position est dans une zone exclue
    def in_excl(pos):
        for start, end in excl_positions:
            if start <= pos < end:
                return True
        return False

    # Chercher toutes les occurences du champ lexical
    for match in pattern_lexical.finditer(text):
        start_pos = match.start()
        if not in_excl(start_pos):
            return True
    return False

In [50]:
# Appliquer sur la colonne
df["repu_match_valide"] = df["Texte_clean"].apply(contains_lexical_outside_excl)

In [51]:
df_match = df[df["repu_match_valide"]]
df_match.shape

(3958, 61)

In [52]:
df_match.to_csv(
    "../data/interim/df_repu.csv",
    index=False,
    quoting=csv.QUOTE_ALL,  # a permis de résoudre le soucis d'écart. Checker
)

In [53]:
# vérif écriture/lecture ok
df_test = pd.read_csv("../data/interim/df_repu.csv")
df_test.shape

(3958, 61)

In [54]:
# ################
# # Pour info si avait bourré sans le span
# # (et exclu des choses qui nous intéressent en vrai)
# ################

# # Version bourrin
# # Champ lexical : True si le texte contient un terme du champ lexical
# lexical_mask = df["Texte_clean"].str.contains(pattern_lexical)

# # Exclusion : True si le texte contient une expression à exclure
# excl_mask = df["Texte_clean"].str.contains(pattern_excl)

# # On garde ceux qui matchent lexicalement mais ne contiennent pas d'exclusion
# df["repu_match_simple"] = lexical_mask & ~excl_mask

# df[df["repu_match_simple"]].shape
